In [55]:
import pandas as pd
import numpy as np
import time
import ciso8601
import json

In [56]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
  sys.path.append(module_path)

In [57]:
from constants import SUBMISSION_TIME, START_TIME, END_TIME, JOB_TYPE, SOURCE, SERVER_TYPE, LOCATION, NAMES

In [58]:
TIME_SLOT_LENGTH = 10 * 60

In [59]:
def safe_parse_date(x: any) -> float:
  try:
    return time.mktime(ciso8601.parse_datetime(str(x)).timetuple())
  except:
    return np.NaN

# Jobs

In [60]:
with open('in/trace-data/cluster_job_log', 'r') as f:
  jobs = json.load(f)

In [61]:
def parse_job(job) -> dict:
  attempts = job['attempts']
  if len(attempts) > 0:
    attempt = attempts[0]
    attempt_info = { START_TIME: safe_parse_date(attempt['start_time']), END_TIME: safe_parse_date(attempt['end_time']) }
  else:
    attempt_info = {}
  return { SUBMISSION_TIME: safe_parse_date(job['submitted_time']), **attempt_info, JOB_TYPE: 0, SOURCE: 0, SERVER_TYPE: 0, LOCATION: 0 }

In [62]:
t = pd.DataFrame([parse_job(job) for job in jobs])
t = t.dropna(axis='rows', subset=[SUBMISSION_TIME])
t = t.sort_values(by=SUBMISSION_TIME)

In [63]:
t.to_csv('out/trace.csv', index=False)

# Servers

In [64]:
servers = pd.read_csv('./in/trace-data/cluster_machine_list', delimiter=',')

In [65]:
servers.groupby([' number of GPUs']).count()

,machineId,single GPU mem
number of GPUs,,
2,321,321
8,231,231
